# PRIO Press Release: Draft Watchlist 

This notebook produces dataframes for analysis, which include predicted fatalities for 2024, January to August 2023 GED sb data, September GED sb data, and October GED sb data. To 


In [10]:
import requests
import numpy as np
import pandas as pd

# VIEWS operations 
from viewser import Queryset, Column


from ingester3.ViewsMonth import ViewsMonth



# from views_forecasts.extensions import * 
# cannot import without certificates, using saved CSV file fetched from data storage. Last monthly run from the API does not have the fixed GED data for Ukraine. See notebook Fetch_Storage in Views_organization/DataExploration to see how to fetch from storage. 

import os
home = os.path.expanduser("~")
desktop = home+'/Desktop' # (Toggle on to save to desktop) 

FileNotFoundError: [Errno 2] No such file or directory: '/Users/alexa.timlick/.postgresql/postgresql.crt'

## Retrieving VIEWS Data

In [46]:
views_oct_fetch = pd.read_csv("October_pred.csv")
views_oct_fetch

,month_id,country_id,step_combined
0,527,1,0.015047
1,527,2,0.016818
2,527,3,0.005570
3,527,4,1.577370
4,527,5,0.011272
...,...,...,...
6871,562,242,1.242629
6872,562,243,0.709711
6873,562,244,0.672846
6874,562,245,234.437940


In [13]:
# find all unique values for month_id to determine the time frame 

print(sorted(views_oct_fetch["month_id"].unique())) # 527-562. 
# 527 = November 2023
# 529 = Jan 2024
# 540 = Dec 2024

[527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562]


## High-Violence Watchlist 

In [48]:
# Filtering for 2024: jan = 529, dec = 540
df_2024_views = views_oct_fetch[(views_oct_fetch['month_id'] >= 529) & (views_oct_fetch['month_id'] <= 540)]
df_2024_views # step_combined is predicted fatalities

,month_id,country_id,step_combined
382,529,1,0.029735
383,529,2,0.036115
384,529,3,0.014260
385,529,4,2.010039
386,529,5,0.010959
...,...,...,...
2669,540,242,0.766481
2670,540,243,0.803236
2671,540,244,0.436316
2672,540,245,143.212069


In [49]:
 # Sum of total predicted fatalities over 2024 for each country
 
df_2024_views_group = df_2024_views.groupby(["country_id"])["step_combined"].sum().reset_index()
df_2024_views_group

,country_id,step_combined
0,1,0.700605
1,2,0.808878
2,3,0.469022
3,4,27.274229
4,5,0.238828
...,...,...
186,242,6.851465
187,243,5.785037
188,244,4.448226
189,245,2202.762328


In [50]:
# Sorting by rank for 2024 
df_2024_rank = df_2024_views_group.sort_values(by="step_combined", ascending=False)

df_2024_rank.reset_index(drop=True, inplace=True)
df_2024_rank.index = df_2024_rank.index + 1

df_2024_rank.index.name = "Ranking"
df_2024_rank

,country_id,step_combined
Ranking,,
1,117,9785.686731
2,220,6973.804442
3,79,5167.577905
4,47,4685.281263
5,149,4227.006836
...,...,...
187,176,0.189181
188,111,0.171796
189,144,0.155312


## Fetching UCDP GED and Candidate Data

In [51]:
def vid2date(i):
    year=str(1980 + i//12)
    month=str(i%12)
    return year+"/"+month

# Printing month_ids for chosen period for reference
print((ViewsMonth.from_year_month(year=2023, month=1)))

ViewsMonth(id=517) #=> year:2023, month:1


In [52]:
StartOfHistory = 517 # January 2023
EndOfHistory = 526 # October 2023, up to end of input data so far

In [53]:
qs_ged_cm = (Queryset("AT_cm_ged_data_no_transformations", "country_month")
             .with_column(Column("year", from_table="month", from_column="year_id"))
             .with_column(Column('month', from_table='month', from_column='month'))
             .with_column(Column('country_name', from_table='country', from_column='name'))
             .with_column(Column("ged_best_sb", from_table="ged2_cm", from_column="ged_sb_best_sum_nokgi"))
             .with_column(Column("ged_best_os", from_table="ged2_cm", from_column="ged_os_best_sum_nokgi"))
             .with_column(Column("ged_best_ns", from_table="ged2_cm", from_column="ged_ns_best_sum_nokgi")))

qs_ged_cm = qs_ged_cm.publish().fetch()

print(f"A dataset with {len(qs_ged_cm.columns)} columns, with "
      f"data between t {min(qs_ged_cm.index.get_level_values(0))} "
      f"and {max(qs_ged_cm.index.get_level_values(0))}. "
      f"({len(np.unique(qs_ged_cm.index.get_level_values(1)))} units)"
     )

 .     A dataset with 6 columns, with data between t 1 and 852. (213 units)


In [54]:
qs_ged_cm

year  month         country_name  ged_best_sb  \
month_id country_id                                                  
1        1           1980      1               Guyana          NaN   
         2           1980      1             Suriname          NaN   
         3           1980      1  Trinidad and Tobago          NaN   
         4           1980      1            Venezuela          NaN   
         5           1980      1                Samoa          NaN   
...                   ...    ...                  ...          ...   
852      242         2050     12             Tanzania          NaN   
         243         2050     12              Morocco          NaN   
         244         2050     12           Mauritania          NaN   
         245         2050     12                Sudan          NaN   
         246         2050     12          South Sudan          NaN   

                     ged_best_os  ged_best_ns  
month_id country_id                            
1        1                   NaN          NaN  
         2                   NaN          NaN  
         3                   NaN          NaN  
         4                   NaN          NaN  
         5                   NaN          NaN  
...                          ...          ...  
852      242                 NaN          NaN  
         243                 NaN          NaN  
         244                 NaN          NaN  
         245                 NaN          NaN  
         246                 NaN          NaN  

[158230 rows x 6 columns]

In [55]:
# From Jan 2023 to Oct 2023
# displays the subset

ged_cm_subset_2023 = qs_ged_cm.query(f"month_id >= {StartOfHistory} and month_id <= {EndOfHistory}")

ged_cm_subset_2023

year  month         country_name  ged_best_sb  \
month_id country_id                                                  
517      1           2023      1               Guyana          0.0   
         2           2023      1             Suriname          0.0   
         3           2023      1  Trinidad and Tobago          0.0   
         4           2023      1            Venezuela          0.0   
         5           2023      1                Samoa          0.0   
...                   ...    ...                  ...          ...   
526      242         2023     10             Tanzania          0.0   
         243         2023     10              Morocco          0.0   
         244         2023     10           Mauritania          0.0   
         245         2023     10                Sudan        166.0   
         246         2023     10          South Sudan          0.0   

                     ged_best_os  ged_best_ns  
month_id country_id                            
517      1                   0.0          0.0  
         2                   0.0          0.0  
         3                   0.0          0.0  
         4                   3.0          0.0  
         5                   0.0          0.0  
...                          ...          ...  
526      242                 0.0          0.0  
         243                 0.0          0.0  
         244                 0.0          0.0  
         245                15.0         16.0  
         246                 0.0          0.0  

[1910 rows x 6 columns]

In [56]:

# Specifcy the columns to keep
columns_to_keep_ged = ["country_name", "ged_best_sb"]

# Filter the DataFrame to only retain the specified columns

ged_df_2023 = ged_cm_subset_2023[columns_to_keep_ged]

# Filter for only jan-aug based on month_id
month_id_jan_aug = list(range(517, 525))
month_id_jan_aug


[517, 518, 519, 520, 521, 522, 523, 524]

In [57]:
# Check the dtype of the index
index_dtype = ged_df_2023.index.dtype

# Print the dtype
print("Index dtype:", index_dtype)

# Check if the index is numeric
is_numeric_index = pd.api.types.is_numeric_dtype(index_dtype)

# Print the result
print("Is numeric index:", is_numeric_index)

# encountering issues filtering the dataframe based on a non-numerical index, for speed to complete the task under the given  timeframe the month_id index is being converted into a column prior to filtering

Index dtype: object
Is numeric index: False


In [98]:
ged_df_months = ged_df_2023.reset_index(level=["month_id", "country_id"])

ged_df_months

,month_id,country_id,country_name,ged_best_sb
0,517,1,Guyana,0.0
1,517,2,Suriname,0.0
2,517,3,Trinidad and Tobago,0.0
3,517,4,Venezuela,0.0
4,517,5,Samoa,0.0
...,...,...,...,...
1905,526,242,Tanzania,0.0
1906,526,243,Morocco,0.0
1907,526,244,Mauritania,0.0
1908,526,245,Sudan,166.0


In [113]:
# creating dataframe for UCDP GED data January-August 2023
ged_df_0123_0823 = ged_df_months[(ged_df_months["month_id"] >= 517) & (ged_df_months["month_id"] <= 524)]
ged_df_0123_0823


,month_id,country_id,country_name,ged_best_sb
0,517,1,Guyana,0.0
1,517,2,Suriname,0.0
2,517,3,Trinidad and Tobago,0.0
3,517,4,Venezuela,0.0
4,517,5,Samoa,0.0
...,...,...,...,...
1523,524,242,Tanzania,0.0
1524,524,243,Morocco,0.0
1525,524,244,Mauritania,0.0
1526,524,245,Sudan,369.0


In [114]:

ged_df_0123_0823["jan_aug_total_ged"] = ged_df_0123_0823.groupby("country_name")["ged_best_sb"].transform("sum")
ged_df_0123_0823

/var/folders/t3/40l96qbj5mlfwpyyhs6h6k8r0000gq/T/ipykernel_51378/1819746763.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ged_df_0123_0823["jan_aug_total_ged"] = ged_df_0123_0823.groupby("country_name")["ged_best_sb"].transform("sum")


,month_id,country_id,country_name,ged_best_sb,jan_aug_total_ged
0,517,1,Guyana,0.0,0.0
1,517,2,Suriname,0.0,0.0
2,517,3,Trinidad and Tobago,0.0,0.0
3,517,4,Venezuela,0.0,11.0
4,517,5,Samoa,0.0,0.0
...,...,...,...,...,...
1523,524,242,Tanzania,0.0,0.0
1524,524,243,Morocco,0.0,1.0
1525,524,244,Mauritania,0.0,6.0
1526,524,245,Sudan,369.0,1843.0


In [116]:
# having each country only once based on total_ged value - removing duplicates so each country is only in the df once based on total_ged, dropping ged_best_sb later

ged_df_0123_0823_unique = ged_df_0123_0823.drop_duplicates(subset=["jan_aug_total_ged", "country_id"])
ged_df_0123_0823_unique

,month_id,country_id,country_name,ged_best_sb,jan_aug_total_ged
0,517,1,Guyana,0.0,0.0
1,517,2,Suriname,0.0,0.0
2,517,3,Trinidad and Tobago,0.0,0.0
3,517,4,Venezuela,0.0,11.0
4,517,5,Samoa,0.0,0.0
...,...,...,...,...,...
186,517,242,Tanzania,0.0,0.0
187,517,243,Morocco,0.0,1.0
188,517,244,Mauritania,0.0,6.0
189,517,245,Sudan,0.0,1843.0


In [101]:
# 525 sept

ged_df_0923 = ged_df_months[(ged_df_months["month_id"] == 525)]
ged_df_0923["sept_total_ged"] = ged_df_0923.groupby("country_name")["ged_best_sb"].transform("sum")
ged_df_0923

/var/folders/t3/40l96qbj5mlfwpyyhs6h6k8r0000gq/T/ipykernel_51378/3527785342.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ged_df_0923["sept_total_ged"] = ged_df_0923.groupby("country_name")["ged_best_sb"].transform("sum")


,month_id,country_id,country_name,ged_best_sb,sept_total_ged
1528,525,1,Guyana,0.0,0.0
1529,525,2,Suriname,0.0,0.0
1530,525,3,Trinidad and Tobago,0.0,0.0
1531,525,4,Venezuela,0.0,0.0
1532,525,5,Samoa,0.0,0.0
...,...,...,...,...,...
1714,525,242,Tanzania,0.0,0.0
1715,525,243,Morocco,0.0,0.0
1716,525,244,Mauritania,0.0,0.0
1717,525,245,Sudan,510.0,510.0


In [117]:
ged_df_0923_unique = ged_df_0923.drop_duplicates(subset=["sept_total_ged", "country_id"])
ged_df_0923_unique

,month_id,country_id,country_name,ged_best_sb,sept_total_ged
1528,525,1,Guyana,0.0,0.0
1529,525,2,Suriname,0.0,0.0
1530,525,3,Trinidad and Tobago,0.0,0.0
1531,525,4,Venezuela,0.0,0.0
1532,525,5,Samoa,0.0,0.0
...,...,...,...,...,...
1714,525,242,Tanzania,0.0,0.0
1715,525,243,Morocco,0.0,0.0
1716,525,244,Mauritania,0.0,0.0
1717,525,245,Sudan,510.0,510.0


In [102]:
# 526 oct
ged_df_1023 = ged_df_months[(ged_df_months["month_id"] == 526)]
ged_df_1023["oct_total_ged"] = ged_df_1023.groupby("country_name")["ged_best_sb"].transform("sum")
ged_df_1023

/var/folders/t3/40l96qbj5mlfwpyyhs6h6k8r0000gq/T/ipykernel_51378/1818295178.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ged_df_1023["oct_total_ged"] = ged_df_1023.groupby("country_name")["ged_best_sb"].transform("sum")


,month_id,country_id,country_name,ged_best_sb,oct_total_ged
1719,526,1,Guyana,0.0,0.0
1720,526,2,Suriname,0.0,0.0
1721,526,3,Trinidad and Tobago,0.0,0.0
1722,526,4,Venezuela,0.0,0.0
1723,526,5,Samoa,0.0,0.0
...,...,...,...,...,...
1905,526,242,Tanzania,0.0,0.0
1906,526,243,Morocco,0.0,0.0
1907,526,244,Mauritania,0.0,0.0
1908,526,245,Sudan,166.0,166.0


In [138]:
ged_df_1023_unique = ged_df_1023.drop_duplicates(subset=["oct_total_ged", "country_id"])
ged_df_1023_unique

,month_id,country_id,country_name,ged_best_sb,oct_total_ged
1719,526,1,Guyana,0.0,0.0
1720,526,2,Suriname,0.0,0.0
1721,526,3,Trinidad and Tobago,0.0,0.0
1722,526,4,Venezuela,0.0,0.0
1723,526,5,Samoa,0.0,0.0
...,...,...,...,...,...
1905,526,242,Tanzania,0.0,0.0
1906,526,243,Morocco,0.0,0.0
1907,526,244,Mauritania,0.0,0.0
1908,526,245,Sudan,166.0,166.0


In [149]:
# merging dataframes based on country_id 
# first merging ged 
ged_common_columns = ["country_id", "country_name"]

final_df_ged = pd.merge(pd.merge(ged_df_0123_0823_unique, ged_df_0923_unique, on=ged_common_columns), ged_df_1023_unique, on=ged_common_columns)
final_df_ged = final_df_ged[final_df_ged.columns.drop(list(final_df_ged.filter(regex="month_id")))]
final_df_ged = final_df_ged[final_df_ged.columns.drop(list(final_df_ged.filter(regex="ged_best_sb")))]
final_df_ged # want to figure out shorter/cleaner code



,country_id,country_name,jan_aug_total_ged,sept_total_ged,oct_total_ged
0,1,Guyana,0.0,0.0,0.0
1,2,Suriname,0.0,0.0,0.0
2,3,Trinidad and Tobago,0.0,0.0,0.0
3,4,Venezuela,11.0,0.0,0.0
4,5,Samoa,0.0,0.0,0.0
...,...,...,...,...,...
186,242,Tanzania,0.0,0.0,0.0
187,243,Morocco,1.0,0.0,0.0
188,244,Mauritania,6.0,0.0,0.0
189,245,Sudan,1843.0,510.0,166.0


In [151]:
final_df = pd.merge(final_df_ged, df_2024_rank, on="country_id") # merging ged and predictions
final_df = final_df.rename(columns={"step_combined": "pred_fatalities_2024"}) #renaming column for understanding

final_df

,country_id,country_name,jan_aug_total_ged,sept_total_ged,oct_total_ged,pred_fatalities_2024
0,1,Guyana,0.0,0.0,0.0,0.700605
1,2,Suriname,0.0,0.0,0.0,0.808878
2,3,Trinidad and Tobago,0.0,0.0,0.0,0.469022
3,4,Venezuela,11.0,0.0,0.0,27.274229
4,5,Samoa,0.0,0.0,0.0,0.238828
...,...,...,...,...,...,...
186,242,Tanzania,0.0,0.0,0.0,6.851465
187,243,Morocco,1.0,0.0,0.0,5.785037
188,244,Mauritania,6.0,0.0,0.0,4.448226
189,245,Sudan,1843.0,510.0,166.0,2202.762328


In [155]:
final_df_rank = final_df.sort_values(by="pred_fatalities_2024", ascending=False) # ranking df based on 2024 predictions, high to low
final_df_rank.reset_index(drop=True, inplace=True)
final_df_rank.index = final_df_rank.index + 1
final_df_rank.index.name = "Ranking"
final_df_rank


,country_id,country_name,jan_aug_total_ged,sept_total_ged,oct_total_ged,pred_fatalities_2024
Ranking,,,,,,
1,117,Ukraine,15402.0,1902.0,2014.0,9785.686731
2,220,Syria,926.0,130.0,316.0,6973.804442
3,79,Nigeria,2542.0,163.0,345.0,5167.577905
4,47,Burkina Faso,2970.0,411.0,425.0,4685.281263
5,149,Myanmar,1426.0,179.0,156.0,4227.006836
...,...,...,...,...,...,...
187,176,Timor Leste,0.0,0.0,0.0,0.189181
188,111,Estonia,0.0,0.0,0.0,0.171796
189,144,Palau,0.0,0.0,0.0,0.155312
